In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import bs4 as BeautifulSoup

from main import *

In [ ]:
# TODO 2: AFEGRIR LA TEMPORADA ALS DATAFRAME DELS LABELS

# TODO 3: Crear dataframe de labels
'''
    3. Blocks
    4. TOV/STL: # TODO: acabar de veure com ho gestionem pq no hi hagi jugades repetides
'''

# TODO 4: Descarregar vídeos, processat i crear datasets

# TODO 5: Pensar què collons analitzaré jejej

# TODO 6: Obtenir 3D tracking data dels vídeos

# TODO 7: Crear/Finetune d'una xarxa multimodal: Vídeos + 3D tracking data

## 1. Scrape labels for FGA, FGM, AST (2p & 3p)

In [ ]:
driver = initialize_driver(season="2023-24")

general_players_table_xpath = '//*[@id="__next"]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[3]/table/'

# Create the empty dataframe for the labels, depending on the type of play
play_type_indices = {"FGA": 11, "REB": 21, "TOV": 23, "STL": 24, "BLK": 25}
play_type = 'FGA'

df_path = '/Users/arnaubarrera/Desktop/MSc Computer Vision/TFM/labeled_plays_NBA/holy_grail/FGA_labels.csv'
labels_df = pd.read_csv(df_path)

for i in range(25, 100):  # Iterate over the list of players

    # Clicar en cierta columna (tipo de jugada) de un jugador en concreto
    try:

        link_xpath = general_players_table_xpath + f'tbody/tr[{i}]/td[{play_type_indices[play_type]}]/a'
        link_element = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, link_xpath))
        )

        link_url = link_element.get_attribute("href")
        driver.get(link_url)

    except Exception as e:
        print(f"Error: {e}")

    video_display_xpath = '//*[@id="vjs_video_3_html5_api"]'

    # Load all the rows in one page before iterating
    dropdown_xpath = '//*[@class="DropDown_select__4pIg9"]'
    option_value = '-1'

    dropdown_element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, dropdown_xpath))
    )
    select = Select(dropdown_element)
    select.select_by_value(option_value)

    rows_number = count_rows_table_plays(driver, table_xpath='//*[@id="__next"]/div[2]/div[2]/div[3]/section/div/div/div[3]/table/tbody')

    for i in range(1, rows_number):  # Iterate over all the plays of a particular player
        line_play = f'//*[@id="__next"]/div[2]/div[2]/div[3]/section/div/div/div[3]/table/tbody/tr[{i}]/td'

        play = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, line_play))
        )

        columns_play = play.find_elements(By.XPATH, line_play)
        
        row_data = []
        for col in columns_play:
            row_data.append(col.text)
        
        play.click()

        # Verify if the shot is assisted
        ast_xpath = '//*[@id="__next"]/div[2]/div[2]/div[3]/section/div/main/section[1]/div/div[2]/h2'

        ast_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, ast_xpath))
        )

        ast_present, assisted_by = extract_assist_info(ast_element.text)
        row_data.extend([ast_present, assisted_by])

        # Video display element
        video_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, video_display_xpath))
        )
        video_src = video_element.get_attribute('src')
        row_data.append(video_src) 

        # La fila se agrega al DataFrame
        labels_df.loc[i - 1] = row_data  
    
    # Actualizar el csv con los nuevos datos
    labels_df.to_csv(df_path, mode='a', index=True, header=False)

    # Cuando se ha guardado, click en la flecha de ir para atrás
    driver.back()
    time.sleep(2)


driver.quit()

## 2.1 Scrape Off REB

In [ ]:
season = "2023-24"
play_type = "OREB"

driver = initialize_driver(season=season)

general_players_table_xpath = '//*[@id="__next"]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[3]/table/'

# Create the empty dataframe for the labels, depending on the type of play
play_type_indices = {"FGA": 11, "OREB": 19, "DREB": 20, "REB": 21, "TOV": 23, "STL": 24, "BLK": 25}

df_path = f'/Users/arnaubarrera/Desktop/MSc Computer Vision/TFM/labeled_plays_NBA/holy_grail/REB/{play_type}_labels.csv'
labels_df = pd.read_csv(df_path)


for i in range(1, 100):  # Iterate over the list of players

    # Player name
    xpath_player_name = f'//*[@id="__next"]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[3]/table/tbody/tr[{i}]/td[2]/a'
    player_name = driver.find_element(By.XPATH, xpath_player_name).text

    # Clicar en cierta columna (tipo de jugada) de un jugador en concreto
    link_xpath = general_players_table_xpath + f'tbody/tr[{i}]/td[{play_type_indices[play_type]}]/a'
    link_element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, link_xpath))
    )

    link_url = link_element.get_attribute("href")
    driver.get(link_url)

    video_display_xpath = '//*[@id="vjs_video_3_html5_api"]'

    # Load all the rows in one page before iterating
    dropdown_xpath = '//*[@class="DropDown_select__4pIg9"]'
    option_value = '-1'

    try:
        dropdown_element = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, dropdown_xpath))
        )
        select = Select(dropdown_element)
        select.select_by_value(option_value)
    except:
        print("There's only one page")


    rows_number = count_rows_table_plays(driver, table_xpath='//*[@id="__next"]/div[2]/div[2]/div[3]/section/div/div/div[3]/table/tbody')

    for i in range(1, rows_number):  # Iterate over all the plays of a particular player
        line_play = f'//*[@id="__next"]/div[2]/div[2]/div[3]/section/div/div/div[3]/table/tbody/tr[{i}]/td'

        play = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, line_play))
        )

        columns_play = play.find_elements(By.XPATH, line_play)
        
        row_data = []
        for col in columns_play:
            row_data.append(col.text)

        row_data.insert(1, player_name)
        row_data.insert(3, 'Off')
        
        play.click()

        # Video display element
        video_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, video_display_xpath))
        )
        video_src = video_element.get_attribute('src')
        row_data.append(video_src) 

        # La fila se agrega al DataFrame
        labels_df.loc[i - 1] = row_data  
    
    # Actualizar el csv con los nuevos datos
    labels_df.to_csv(df_path, mode='a', index=True, header=False)

    # Cuando se ha guardado, click en la flecha de ir para atrás
    driver.back()
    time.sleep(2)


driver.quit()

Cookies banner closed.
There's only one page
There's only one page
There's only one page
There's only one page
There's only one page
There's only one page
There's only one page
There's only one page
There's only one page
There's only one page
There's only one page
There's only one page
There's only one page
There's only one page
There's only one page
There's only one page
There's only one page
There's only one page
There's only one page
There's only one page
There's only one page
There's only one page
There's only one page


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="__next"]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[3]/table/tbody/tr[51]/td[2]/a"}
  (Session info: chrome=131.0.6778.265); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x000000010518e138 cxxbridge1$str$ptr + 3653888
1   chromedriver                        0x0000000105186988 cxxbridge1$str$ptr + 3623248
2   chromedriver                        0x0000000104bec968 cxxbridge1$string$len + 89228
3   chromedriver                        0x0000000104c30d4c cxxbridge1$string$len + 368752
4   chromedriver                        0x0000000104c6a4f0 cxxbridge1$string$len + 604180
5   chromedriver                        0x0000000104c25564 cxxbridge1$string$len + 321672
6   chromedriver                        0x0000000104c261b4 cxxbr

## 2.2 Scrape Def REB

In [15]:
season = "2023-24"
play_type = "DREB"

driver = initialize_driver(season=season)

general_players_table_xpath = '//*[@id="__next"]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[3]/table/'

# Create the empty dataframe for the labels, depending on the type of play
play_type_indices = {"FGA": 11, "OREB": 19, "DREB": 20, "REB": 21, "TOV": 23, "STL": 24, "BLK": 25}

df_path = f'/Users/arnaubarrera/Desktop/MSc Computer Vision/TFM/labeled_plays_NBA/holy_grail/REB/{play_type}_labels.csv'
labels_df = pd.read_csv(df_path)

for i in range(1, 100):  # Iterate over the list of players

    # Player name
    xpath_player_name = f'//*[@id="__next"]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[3]/table/tbody/tr[{i}]/td[2]/a'
    player_name = driver.find_element(By.XPATH, xpath_player_name).text

    # Clicar en cierta columna (tipo de jugada) de un jugador en concreto
    link_xpath = general_players_table_xpath + f'tbody/tr[{i}]/td[{play_type_indices[play_type]}]/a'
    link_element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, link_xpath))
    )

    link_url = link_element.get_attribute("href")
    driver.get(link_url)

    video_display_xpath = '//*[@id="vjs_video_3_html5_api"]'

    # Load all the rows in one page before iterating
    dropdown_xpath = '//*[@class="DropDown_select__4pIg9"]'
    option_value = '-1'

    try:
        dropdown_element = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, dropdown_xpath))
        )
        select = Select(dropdown_element)
        select.select_by_value(option_value)
    except:
        print("There's only one page")
        

    rows_number = count_rows_table_plays(driver, table_xpath='//*[@id="__next"]/div[2]/div[2]/div[3]/section/div/div/div[3]/table/tbody')

    for i in range(1, rows_number):  # Iterate over all the plays of a particular player
        line_play = f'//*[@id="__next"]/div[2]/div[2]/div[3]/section/div/div/div[3]/table/tbody/tr[{i}]/td'

        play = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, line_play))
        )

        columns_play = play.find_elements(By.XPATH, line_play)
        
        row_data = []
        for col in columns_play:
            row_data.append(col.text)

        row_data.insert(1, player_name)
        row_data.insert(3, 'Off')
        
        play.click()

        # Video display element
        video_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, video_display_xpath))
        )
        video_src = video_element.get_attribute('src')
        row_data.append(video_src) 

        # La fila se agrega al DataFrame
        labels_df.loc[i - 1] = row_data  
    
    # Actualizar el csv con los nuevos datos
    labels_df.to_csv(df_path, mode='a', index=True, header=False)

    # Cuando se ha guardado, click en la flecha de ir para atrás
    driver.back()
    time.sleep(2)


driver.quit()

Cookies banner closed.


TimeoutException: Message: 
Stacktrace:
0   chromedriver                        0x0000000102c42138 cxxbridge1$str$ptr + 3653888
1   chromedriver                        0x0000000102c3a988 cxxbridge1$str$ptr + 3623248
2   chromedriver                        0x00000001026a0968 cxxbridge1$string$len + 89228
3   chromedriver                        0x00000001026e4d4c cxxbridge1$string$len + 368752
4   chromedriver                        0x000000010271e4f0 cxxbridge1$string$len + 604180
5   chromedriver                        0x00000001026d9564 cxxbridge1$string$len + 321672
6   chromedriver                        0x00000001026da1b4 cxxbridge1$string$len + 324824
7   chromedriver                        0x0000000102c0cfc0 cxxbridge1$str$ptr + 3436424
8   chromedriver                        0x0000000102c102dc cxxbridge1$str$ptr + 3449508
9   chromedriver                        0x0000000102bf3e60 cxxbridge1$str$ptr + 3333672
10  chromedriver                        0x0000000102c10b9c cxxbridge1$s

## 3. Download videos

In [19]:
df_path = '/Users/arnaubarrera/Desktop/MSc Computer Vision/TFM/labeled_plays_NBA/holy_grail/REB/OREB_labels.csv'
df = pd.read_csv(df_path)
#df = clean_missing_video_data(df)

In [ ]:
download_videos_from_dataframe(df, name='FGA', output_dir=df_path, max_workers=32)

In [20]:
tipos_unicos = df['PLAYER'].unique()

In [21]:
len(tipos_unicos)

50

In [11]:
print("Tipos de datos distintos:", tipos_unicos)
print("Cantidad de tipos distintos:", len(tipos_unicos))

Tipos de datos distintos: [<class 'str'>]
Cantidad de tipos distintos: 1
